In [2]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy

from utils import solr
from utils.logger import logger as log

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [34]:
df = pd.read_pickle('./data/gene_allele_all.pkl')

df = df.apply(lambda x: x.str.strip().str.lower())

df = df.drop(['zygosity'], axis=1)

df = df.drop_duplicates()

df_gene_allele_all = df.copy()

impc_genes = df['gene_symbol'].drop_duplicates().str.strip().str.lower()

impc_genes.shape

(8416,)

In [4]:
with open('./data/uniprot-compressed_true_download_true_format_json_query__28_28taxono-2023.02.05-17.45.13.08.json', 'r') as f:
    data = json.load(f)

In [5]:
records = []
results = data['results']
for result in results:
    primary_accession = result['primaryAccession']
    uniprotkb_id = result['uniProtkbId']
    if 'sequence' in result:
        sequence = result['sequence']['value']
        if 'genes' in result and len(result['genes']) != 0:
            for gene in result['genes']:
                gene_names = []
                if 'geneName' in gene:
                    gene_names.append(gene['geneName'])
                if 'synonyms' in gene:
                    gene_names.extend(gene['synonyms'])
                if 'orfNames' in gene:
                    gene_names.extend(gene['orfNames'])
                for gene_name in gene_names:
                    if 'value' in gene_name:
                        gene_symbol = gene_name['value']
                        records.append({
                            'gene_symbol': gene_symbol,
                            'primary_accession': primary_accession,
                            'uniprotkb_id': uniprotkb_id,
                            'sequence': sequence
                        })

In [ ]:
df = pd.DataFrame(records)

df.to_pickle('./data/gene_symbol_protein_sequence_all.pkl')

_df_gene_symbol_protein_sequence_all = df.copy()

In [27]:
df = _df_gene_symbol_protein_sequence_all.copy()

# df['gene_symbol'] = df['gene_symbol'].str.strip().str.lower()

# df = df.loc[df['gene_symbol'].isin(impc_genes)]

# df = df.groupby(['gene_symbol']).filter(lambda x: x['sequence'].drop_duplicates().shape[0] > 1)

# df['gene_symbol'].drop_duplicates().shape

# df = df.groupby(['sequence']).filter(lambda x: x['gene_symbol'].drop_duplicates().shape[0] > 1)

df

,gene_symbol,primary_accession,uniprotkb_id,sequence
600,ccnb1ip1,D3Z3K2,CIP1_MOUSE,MSLCEDMLLCNYRKCRIKLSGYAWVTACSHIFCDQHGSGEFSRSPAICPACNSTLSGKLDIVRTELSPSEEYKAMVLAGLRPEVVLDISSRALAFWTYQVHQERLYQEYNFSKAENHLKQMEKMYMQQIQSKNIELTSMKGEVISMKKVLEEYKKKFSDISEKLMERNRQYQKLQGLYDSLRLRNITIASQEGSLEPGMIPQSGVFGFPPGNNSKFSLDHIPVGNQGGGDEDVQFRPFFVCSPTAPEPINNFFSFASPSHEAEQQVCSRAFKAKRI
603,mei4,D3Z3K2,CIP1_MOUSE,MSLCEDMLLCNYRKCRIKLSGYAWVTACSHIFCDQHGSGEFSRSPAICPACNSTLSGKLDIVRTELSPSEEYKAMVLAGLRPEVVLDISSRALAFWTYQVHQERLYQEYNFSKAENHLKQMEKMYMQQIQSKNIELTSMKGEVISMKKVLEEYKKKFSDISEKLMERNRQYQKLQGLYDSLRLRNITIASQEGSLEPGMIPQSGVFGFPPGNNSKFSLDHIPVGNQGGGDEDVQFRPFFVCSPTAPEPINNFFSFASPSHEAEQQVCSRAFKAKRI
827,arid4a,F8VPQ2,ARI4A_MOUSE,MKAADEPAYLTVGTDVSAKYRGAFCEAKIKTVKRLVKVKVLLKQDNTTQLVQDDQVKGPLRVGAIVETRTSDGSIQEAIISKLTDASWYTVVFDDGDERTLRRTSLCLKGERHFAESETLDQLPLTNPEHFGTPVIAKKTNRGRRSSLPITEDEKEEESSEEEDEDKRRLNDELLGKVVSVASTAESTGWYPALVVSPSCNDDVTVKKDQCLVRSFIDSKFYSIARKDIKELDILTLPESELCARPGLRRASVFLKGRIVPDNWKMDISEILESSSSDDEECPAEEHEEEKEKEAKKEEEELPEEELDPEERDNFLQQLYKFMEDRGTPINKPPVLGYKDLNLFKLFRLVYHQGGCGNIDSGAVWKQIYMDLGIPILNSAASYNVKTAYRKYLYGFEEYCRSANIQFRTIHHHEPKVKEEKKDFEDSMDEALKEAPEMPLLDVKSEPEENTDSNSESDREDTELKSPRGRRKIVRDANCIKKEIEEEKIEDKFLRDDLENKDAGDDDDDGDPAAKREHELLFGRKSTPKNKEKKIKKPEDSERDSDEEEEKSQEREETESRCDSEGEDEEDDTEPCLTGTKVKVKYGRGKTQKIYEASIKSTEMDDGEILYLVHYYGWNVRYDEWVKADRIIWPLDKGGPKKKQKKKVKNKEDSEKDEKRDEERQKSKRGRPPLKSTFSPNMPYSLSKTSNSEGKSDSCSSDSEADDQLEKSSGGEDLSPDVKEELEKNENAHDDKLDEENPKIVHISKENDRTQAQPSDTLTVEAGDSDQIVHIFGDKVDQVEEFKKQVEKSPKGKGRRSKTKDLSLELIKISPFGQEEAGSEAHGDVHSLEFSSLECKNFSSTEDDIDPYEKEKKLKRKILGQQSPEKKLRLDNGMEMTTGVSQERSDDGAGAEGMKGAHVEQHFETEGEGMPSLTAEPDQGLQELTSEKSDSPAEEEPVHTPLKEEEDAMPLIGPETLVCHEVDLDDLDEKDKTSIEDVVVEGSESNSLASVPPALPPVAQHNFSVASPLTLSQDESRSIKSESDITIEVDSIAEESQEGLCERESANGFEASVASGACSIIAHERESREKGQKRPSDGNSGLIAKKQKRTPKRTSAAAKTEKNGAGQSSDSEDLPAMDSSSNCTPVKRLTLPKSQKLPRSPARTSPHIKDAEKEKHREKHPNSSPRTYKWSFQLNELDNMNSTERISFLQEKLQEIRKYYMSLKSEVATIDRRRKRLKKKDREVSHAGASMSSASSDTGMSPSSSSPPQNVLAVECR
829,rbp1,F8VPQ2,ARI4A_MOUSE,MKAADEPAYLTVGTDVSAKYRGAFCEAKIKTVKRLVKVKVLLKQDNTTQLVQDDQVKGPLRVGAIVETRTSDGSIQEAIISKLTDASWYTVVFDDGDERTLRRTSLCLKGERHFAESETLDQLPLTNPEHFGTPVIAKKTNRGRRSSLPITEDEKEEESSEEEDEDKRRLNDELLGKVVSVASTAESTGWYPALVVSPSCNDDVTVKKDQCLVRSFIDSKFYSIARKDIKELDILTLPESELCARPGLRRASVFLKGRIVPDNWKMDISEILESSSSDDEECPAEEHEEEKEKEAKKEEEELPEEELDPEERDNFLQQLYKFMEDRGTPINKPPVLGYKDLNLFKLFRLVYHQGGCGNIDSGAVWKQIYMDLGIPILNSAASYNVKTAYRKYLYGFEEYCRSANIQFRTIHHHEPKVKEEKKDFEDSMDEALKEAPEMPLLDVKSEPEENTDSNSESDREDTELKSPRGRRKIVRDANCIKKEIEEEKIEDKFLRDDLENKDAGDDDDDGDPAAKREHELLFGRKSTPKNKEKKIKKPEDSERDSDEEEEKSQEREETESRCDSEGEDEEDDTEPCLTGTKVKVKYGRGKTQKIYEASIKSTEMDDGEILYLVHYYGWNVRYDEWVKADRIIWPLDKGGPKKKQKKKVKNKEDSEKDEKRDEERQKSKRGRPPLKSTFSPNMPYSLSKTSNSEGKSDSCSSDSEADDQLEKSSGGEDLSPDVKEELEKNENAHDDKLDEENPKIVHISKENDRTQAQPSDTLTVEAGDSDQIVHIFGDKVDQVEEFKKQVEKSPKGKGRRSKTKDLSLELIKISPFGQEEAGSEAHGDVHSLEFSSLECKNFSSTEDDIDPYEKEKKLKRKILGQQSPEKKLRLDNGMEMTTGVSQERSDDGAGAEGMKGAHVEQHFETEGEGMPSLTAEPDQGLQELTSEKSDSPAEEEPVHTPLKEEEDAMPLIGPETLVCHEVDLDDLDEKDKTSIEDVVVEGSESNSLASVPPALPPVAQHNFSVASPLTLSQDESRSIKSESDITIEVDSIAEESQEGLCERESANGFEASVASGACSIIAHERESREKGQKRPSDGNSGLIAKKQKRTPKRTSAAAKTEKNGAGQSSDSEDLPAMDSSSNCTPVKRLTLPKSQKLPRSPARTSPHIKDAEKEKHREKHPNSSPRTYKWSFQLNELDNMNSTERISFLQEKLQEIRKYYMSLKSEVATIDRRRKRLKKKDREVSHAGASMSSASSDTGMSPSSSSPPQNVLAVECR
1128,ncoa3,O09000,NCOA3_MOUSE,MSGLGESSLDPLAAESRKRKLPCDAPGQGLVYSGEKWRREQESKYIEELAELISANLSDIDNFNVKPDKCAILKETVRQIRQIKEQGKTISSDDDVQKADVSSTGQGVIDKDSLGPLLLQALDGFLFVVNRDGNIVFVSENVTQYLQYKQEDLVNTSVYSILHEPRRKDFLNTYQNPQLMEFLGLMRTRDKKAPYILIVRMLMKTHDILEDVNASPETRQRYETMQCFALSQPRAMLEEGEDLQCCMICVARRVTAPFPSSPESFITRHDLSGKVVNIDTNSLRSSMRPGFEDIIRRCIQRFFSLNDGQSWSQKRHYQEAYVHGHAETPVYRFSLADGTIVSAQTKSKLFRNPVTNDRHGFISTHFLQREQNGYRPNPIPQDKGIRPPAAGCGVSMSPNQNVQMMGSRTYGVPDPSNTGQMGGARYGASSSVASLTPGQSLQSPSSYQNSSYGLSMSSPPHGSPGLGPNQQNIMISPRNRGSPKMASHQFSPAAGAHSPMGPSGNTGSHSFSSSSLSALQAISEGVGTSLLSTLSSPGPKLDNSPNMNISQPSKVSGQDSKSPLGLYCEQNPVESSVCQSNSRDPQVKKESKESSGEVSETPRGPLESKGHKKLLQLLTCSSDDRGHSSLTNSPLDPNCKDSSVSVTSPSGVSSSTSGTVSSTSNVHGSLLQEKHRILHKLLQNGNSPAEVAKITAEATGKDTSSTASCGEGTTRQEQLSPKK

In [36]:
# df = pd.read_csv('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled.csv')

# df = df.merge(df_gene_allele_all, how='left', on='allele_accession_id')

# df['gene_symbol'].drop_duplicates().shape

(7882,)